### 乙烷: 读取文件，端口和坐标变换
- 注意:mBuild期望所有的距离单位都是纳米。
- 在本例中，将介绍从文件中读取分子组件，介绍Ports的概念，并开始使用一些坐标变换。

In [1]:
import mbuild as mb

/home/cup/softwares/miniconda/miniconda3/envs/py3.11/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/home/cup/softwares/miniconda/miniconda3/envs/py3.11/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/home/cup/softwares/miniconda/miniconda3/envs/py3.11/lib/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
/home/cup/softwares/miniconda/miniconda3/envs/py3.11/lib/python3.11/site-packages/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]


- 手动为化合物添加粒子和键是有点麻烦的。
- 创建小型可重用组件(如甲基、胺或单体)的最简单方法是使用Avogadro等软件手绘它们，
- 并将它们导出为.pdb或.mol2文件(该文件应包含连接信息)。

In [6]:
# 读取甲基
ch3 = mb.load('./data/ch3.pdb')
# ch3 = mb.load('./data/methane.pdb')
ch3.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# 现在我们用第一个坐标变换把甲基放在碳原子的中心
ch3.translate(-ch3[0].pos)  # Move carbon to origin.

In [8]:
ch3.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
# 为了在mBuild中连接Compound，我们使用了一种特殊类型的Compound: Port
# Port是一种含有两组4个“ghost”粒子的化合物，这些粒子有助于形成键。
# 此外，Port有一个anchor属性，通常指向Port应该关联的粒子
# 在甲基中，Port应该被固定在碳原子上这样我们就可以和这个碳形成键了
port = mb.Port(anchor=ch3[0])
ch3.add(port, label='up')

In [11]:
# 将端口放置在大约半个C-C键长。
ch3['up'].translate([0, -0.07, 0])

In [13]:
ch3.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
# 现在我们把甲基包装到一个python类中，
# 这样我们就可以把它作为一个组件来重用，以便以后构建更复杂的分子。
class CH3(mb.Compound):
    def __init__(self):
        super(CH3, self).__init__()

        mb.load('./data/ch3.pdb', compound=self)
        self.translate(-self[0].pos)  # Move carbon to origin.

        port = mb.Port(anchor=self[0])
        self.add(port, label='up')
        # Place the port at approximately half a C-C bond length.
        self['up'].translate([0, -0.07, 0])


In [22]:
# 当两个端口连接时，它们被迫在空间上重叠，
# 并且它们的母体化合物被旋转和平移相同的量。
# 把它们粘在一起产生乙烷:
ethane = mb.Compound()
ethane.add(CH3(), label="methyl_1")
ethane.add(CH3(), label="methyl_2")

In [23]:
# force_overlap()函数接受一个Compound，
# 然后旋转并转换它，使另外两个Compound重叠。
mb.force_overlap(move_this=ethane['methyl_1'],
                         from_positions=ethane['methyl_1']['up'],
                         to_positions=ethane['methyl_2']['up'])

In [24]:
ethane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [25]:
ethane.visualize(show_ports=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [26]:
# 类似地，如果我们想使乙烷成为可重用组件，我们需要将其包装到python类中。
class Ethane(mb.Compound):
    def __init__(self):
        super(Ethane, self).__init__()

        self.add(CH3(), label="methyl_1")
        self.add(CH3(), label="methyl_2")
        mb.force_overlap(move_this=self['methyl_1'],
                         from_positions=self['methyl_1']['up'],
                         to_positions=self['methyl_2']['up'])


In [27]:
ethane = Ethane()
ethane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol